In [1]:
# give the number of the latest FPL round
latest_gameweek = 38

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
import json
import requests
import xgboost
import pickle
import shap
from natsort import natsorted

# allow more data columns to be shown than by default
pd.set_option('max_columns',500)

In [3]:
filepath = Path('../data/fpl/data_week' + str(latest_gameweek) + str('.csv'))
players = pd.read_csv(filepath, index_col=0)
players

,adjusted points,adjusted points per game,assists,assists_week1,assists_week10,assists_week11,assists_week12,assists_week13,assists_week14,assists_week15,assists_week16,assists_week17,assists_week18,assists_week19,assists_week2,assists_week20,assists_week21,assists_week22,assists_week23,assists_week24,assists_week25,assists_week26,assists_week27,assists_week28,assists_week29,assists_week3,assists_week30,assists_week31,assists_week32,assists_week33,assists_week34,assists_week35,assists_week36,assists_week37,assists_week38,assists_week4,assists_week5,assists_week6,assists_week7,assists_week8,assists_week9,bonus,bonus_week1,bonus_week10,bonus_week11,bonus_week12,bonus_week13,bonus_week14,bonus_week15,bonus_week16,bonus_week17,bonus_week18,bonus_week19,bonus_week2,bonus_week20,bonus_week21,bonus_week22,bonus_week23,bonus_week24,bonus_week25,bonus_week26,bonus_week27,bonus_week28,bonus_week29,bonus_week3,bonus_week30,bonus_week31,bonus_week32,bonus_week33,bonus_week34,bonus_week35,bonus_week36,bonus_week37,bonus_week38,bonus_week4,bonus_week5,bonus_week6,bonus_week7,bonus_week8,bonus_week9,bps,chance_of_playing_next_round,chance_of_playing_this_round,clean_sheet_points,clean_sheets,cleansheet_week1,cleansheet_week10,cleansheet_week11,cleansheet_week12,cleansheet_week13,cleansheet_week14,cleansheet_week15,cleansheet_week16,cleansheet_week17,cleansheet_week18,cleansheet_week19,cleansheet_week2,cleansheet_week20,cleansheet_week21,cleansheet_week22,cleansheet_week23,cleansheet_week24,cleansheet_week25,cleansheet_week26,cleansheet_week27,cleansheet_week28,cleansheet_week29,cleansheet_week3,cleansheet_week30,cleansheet_week31,cleansheet_week32,cleansheet_week33,cleansheet_week34,cleansheet_week35,cleansheet_week36,cleansheet_week37,cleansheet_week38,cleansheet_week4,cleansheet_week5,cleansheet_week6,cleansheet_week7,cleansheet_week8,cleansheet_week9,code,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,creativity,creativity_rank,creativity_rank_type,direct_freekicks_order,direct_freekicks_text,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,form 10,form 15,form 20,form 5,games played,gameweek,gameweek 10 prediction,gameweek 11 prediction,gameweek 12 prediction,gameweek 13 prediction,gameweek 14 prediction,gameweek 15 prediction,gameweek 16 prediction,gameweek 17 prediction,gameweek 18 prediction,gameweek 19 prediction,gameweek 2 prediction,gameweek 20 prediction,gameweek 21 prediction,gameweek 22 prediction,gameweek 23 prediction,gameweek 24 prediction,gameweek 25 prediction,gameweek 26 prediction,gameweek 27 prediction,gameweek 28 prediction,gameweek 29 prediction,gameweek 3 prediction,gameweek 30 prediction,gameweek 31 prediction,gameweek 32 prediction,gameweek 33 prediction,gameweek 34 prediction,gameweek 35 prediction,gameweek 36 prediction,gameweek 37 prediction,gameweek 38 prediction,gameweek 39 prediction,gameweek 4 prediction,gameweek 40 prediction,gameweek 41 prediction,gameweek 42 prediction,gameweek 43 prediction,gameweek 44 prediction,gameweek 45 prediction,gameweek 46 prediction,gameweek 47 prediction,gameweek 5 prediction,gameweek 6 prediction,gameweek 7 prediction,gameweek 8 prediction,gameweek 9 prediction,goals_conceded,goals_scored,goals_week1,goals_week10,goals_week11,goals_week12,goals_week13,goals_week14,goals_week15,goals_week16,goals_week17,goals_week18,goals_week19,goals_week2,goals_week20,goals_week21,goals_week22,goals_week23,goals_week24,goals_week25,goals_week26,goals_week27,goals_week28,goals_week29,goals_week3,goals_week30,goals_week31,goals_week32,goals_week33,goals_week34,goals_week35,goals_week36,goals_week37,goals_week38,goals_week4,goals_week5,goals_week6,goals_week7,goals_week8,goals_week9,ict_index,ict_index_rank,ict_index_rank_type,in_dreamteam,influence,influence_rank,influence_rank_type,minutes,news,news_added,now_cost,own_goals,penalties_missed,penalties_order,penalti

In [4]:
teams = ['Arsenal', 'Aston Villa', 'Brentford', 'Brighton', 'Burnley', 'Chelsea', 'Crystal Palace', 'Everton',
        'Leicester City', 'Leeds United', 'Liverpool', 'Manchester City', 'Manchester Utd', 'Newcastle Utd', 
        'Norwich City', 'Southampton', 'Tottenham', 'Watford', 'West Ham', 'Wolves']

In [5]:
# get FPL fixtures data
fpl_fixtures_data = json.loads(requests.get('https://fantasy.premierleague.com/api/fixtures/').text)
fpl_fixtures = pd.DataFrame(fpl_fixtures_data)
fpl_fixtures['home team'] = [teams[i] for i in fpl_fixtures['team_h']-1]
fpl_fixtures['away team'] = [teams[i] for i in fpl_fixtures['team_a']-1]
filepath = Path('../data/fpl/fixtures.csv')
fpl_fixtures.to_csv(filepath)
fpl_fixtures

,code,event,finished,finished_provisional,id,kickoff_time,minutes,provisional_start_time,started,team_a,team_a_score,team_h,team_h_score,stats,team_h_difficulty,team_a_difficulty,pulse_id,home team,away team
0,2210271,1,True,True,1,2021-08-13T19:00:00Z,90,False,True,1,0,3,2,"[{'identifier': 'goals_scored', 'a': [], 'h': ...",4,2,66342,Brentford,Arsenal
1,2210276,1,True,True,6,2021-08-14T11:30:00Z,90,False,True,10,1,13,5,"[{'identifier': 'goals_scored', 'a': [{'value'...",2,4,66347,Manchester Utd,Leeds United
2,2210272,1,True,True,2,2021-08-14T14:00:00Z,90,False,True,4,2,5,1,"[{'identifier': 'goals_scored', 'a': [{'value'...",2,2,66343,Burnley,Brighton
3,2210273,1,True,True,3,2021-08-14T14:00:00Z,90,False,True,7,0,6,3,"[{'identifier': 'goals_scored', 'a': [], 'h': ...",2,4,66344,Chelsea,Crystal Palace
4,2210274,1,True,True,4,2021-08-14T14:00:00Z,90,False,True,16,1,8,3,"[{'identifier': 'goals_scored', 'a': [{'value'...",2,2,66345,Everton,Southampton
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,2210646,38,True,True,376,2022-05-22T15:00:00Z,90,False,True,13,0,7,1,"[{'identifier': 'goals_scored', 'a': [], 'h': ...",3,2,66717,Crystal Palace,Manchester Utd
376,2210647,38,True,True,377,2022-05-22T15:00:00Z,90,False,True,16,1,9,4,"[{'identifier': 'goals_scored', 'a': [{'value'...",2,3,66718,Leicester City,Southampton
377,2210648,38,True,True,378,2022-05-22T15:00:00Z,90,False,True,20,1,11,3,"[{'identifier': 'goals_scored', 'a': [{'value'...",3,5,66719,Liverpool,Wolves
378,2210649,38,True,True,379,2022-05-22T15:00:00Z,90,False,True,2,2,12,3,"[{'identifier': 'goals_scored', 'a': [{'value'...",2,5,66720,Manchester City,Aston Villa


In [6]:
filepath = Path(f'../data/fbref/team_stats_week{latest_gameweek}.csv')
team_stats = pd.read_csv(filepath, index_col=0)
team_stats

,Rk,MP,W,D,L,GF,GA,GD,Pts,xG,xGA,xGD,xGD/90,Attendance,Top Team Scorer,Goalkeeper,Notes,xG per game,xGA per game,probability no goals allowed
Squad,,,,,,,,,,,,,,,,,,,,
Manchester City,1,38,29,6,3,99,26,73,93,89.0,27.2,61.9,1.63,52774,Kevin De Bruyne - 15,Ederson,NaN,2.342105,0.715789,0.488806
Liverpool,2,38,28,8,2,94,26,68,92,89.2,34.2,55.0,1.45,53367,Mohamed Salah - 23,Alisson,NaN,2.347368,0.900000,0.406570
Chelsea,3,38,21,11,6,76,33,43,74,67.2,36.0,31.3,0.82,36443,Mason Mount - 11,Edouard Mendy,NaN,1.768421,0.947368,0.387760
Tottenham,4,38,22,5,11,69,40,29,71,65.1,39.4,25.8,0.68,56523,Son Heung-min - 23,Hugo Lloris,NaN,1.713158,1.036842,0.354573
Arsenal,5,38,22,3,13,61,48,13,69,59.9,47.1,12.8,0.34,59665,Bukayo Saka - 11,Aaron Ramsdale,NaN,1.576316,1.239474,0.289537
Manchester Utd,6,38,16,10,12,57,57,0,58,54.8,54.5,0.3,0.01,73150,Cristiano Ronaldo - 18,David de Gea,NaN,1.442105,1.434211,0.238303
West Ham,7,38,16,8,14,60,51,9,56,49.9,51.0,-1.1,-0.03,58370,Jarrod Bowen - 12,Łukasz Fabiański,NaN,1.313158,1.342105,0.261295
Leicester City,8,38,14,10,14,62,59,3,52,50.8,61.0,-10.1,-0.27,32493,Jamie Vardy - 15,Kasper Schmeichel,NaN,1.336842,1.605263,0.200837
Brighton,9,38,12,15,11,42,44,-2,51,46.0,45.6,0.4,0.01,30966,"Leandro Trossard, Neal Maupay - 8",Robert Sánchez,NaN,1.210526,1.200000,0.301194


In [7]:
# upload model for making predictions
path = Path('../models/xgboost_predictor_v4_weightsAndBonus.pickle')
with open(path,'rb') as f:
    model = pickle.load(f)

In [8]:
# pre-allocate for model input
predictions = pd.DataFrame(index=players.index.copy())
input_columns = ['goalkeeper', 'defender', 'midfielder',
       'forward', 'xG90', 'xA90', 'xGA90', 'bonus', 'xPoints90',
       'games played', 'form 5', 'games_form5', 'form 10', 'games form10',
       'team xG per 90', 'team xGA per 90', 'home', 'opponent xG per 90',
       'opponent xGA per 90']
predictions[input_columns] = np.nan
predictions

,goalkeeper,defender,midfielder,forward,xG90,xA90,xGA90,bonus,xPoints90,games played,form 5,games_form5,form 10,games form10,team xG per 90,team xGA per 90,home,opponent xG per 90,opponent xGA per 90
id,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
661,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
685,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
686,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# run predictions for all players for a given time horizon
horizon = 10
#horizon = 38-latest_gameweek
output_columns = [f'gameweek {i} prediction' for i in range(latest_gameweek+1,latest_gameweek+horizon+1)]
xGA_columns = [column for column in players.columns if 'xGA_week' in column]
xGA_columns = natsorted(xGA_columns)
xPoints_columns = [column for column in players.columns if 'xPoints week' in column]
xPoints_columns = natsorted(xPoints_columns)
for player_ix in players.index:
    
    # collect base info
    if players.loc[player_ix,'element_type'] == 1:
        predictions.loc[player_ix,'goalkeeper'] = 1
    elif players.loc[player_ix, 'element_type'] == 2:
        predictions.loc[player_ix,'defender'] = 1
    elif players.loc[player_ix, 'element_type'] == 3:
        predictions.loc[player_ix,'midfielder'] = 1
    elif players.loc[player_ix, 'element_type'] == 4:
        predictions.loc[player_ix,'forward'] = 1

    if players.loc[player_ix,'games played'] > 0:
        predictions.loc[player_ix, 'xG90'] = players.loc[player_ix,'xG'] / players.loc[player_ix,'games played']
        predictions.loc[player_ix, 'xA90'] = players.loc[player_ix,'xA'] / players.loc[player_ix,'games played']
        predictions.loc[player_ix, 'xGA90'] = players.loc[player_ix,xGA_columns].sum() / \
                                          players.loc[player_ix,'games played']                                        
        predictions.loc[player_ix, 'bonus'] = players.loc[player_ix, 'bps']
        predictions.loc[player_ix, 'xPoints90'] = players.loc[player_ix,'adjusted points per game']
        predictions.loc[player_ix, 'games played'] = players.loc[player_ix, 'games played']
        
        form5_points = players.loc[player_ix, xPoints_columns[-5:]].sum()
        form5_games = players.loc[player_ix, xPoints_columns[-5:]].notnull().sum()
        predictions.loc[player_ix, 'form 5'] = form5_points
        predictions.loc[player_ix, 'games_form5'] = form5_games
        
        form10_points = players.loc[player_ix, xPoints_columns[-10:]].sum()
        form10_games = players.loc[player_ix, xPoints_columns[-10:]].notnull().sum()
        predictions.loc[player_ix, 'form 10'] = form10_points
        predictions.loc[player_ix, 'games form10'] = form10_games
    else:
        predictions.loc[player_ix, 'xG90'] = 0
        predictions.loc[player_ix, 'xA90'] = 0
        predictions.loc[player_ix, 'xGA90'] = 0                       
        predictions.loc[player_ix, 'bonus'] = 0
        predictions.loc[player_ix, 'xPoints90'] = 0
        predictions.loc[player_ix, 'games played'] = 0
        predictions.loc[player_ix, 'form 5'] = 0
        predictions.loc[player_ix, 'games_form5'] = 0
        predictions.loc[player_ix, 'form 10'] = 0
        predictions.loc[player_ix, 'games form10'] = 0
        
    team = players.loc[player_ix, 'team_name']
    # team xG per 90 
    team_xG90 = team_stats.loc[team,'xG'] / team_stats.loc[team,'MP']
    predictions.loc[player_ix, 'team xG per 90'] = team_xG90
    # team xGA per 90 
    team_xGA90 = team_stats.loc[team,'xGA'] / team_stats.loc[team,'MP']
    predictions.loc[player_ix, 'team xGA per 90'] = team_xGA90
    
    for gameweek in range(latest_gameweek+1,latest_gameweek+horizon+1):
        
        players.loc[player_ix, f'gameweek {gameweek} prediction'] = 0
        
        # home games
        home_fixtures = fpl_fixtures[(fpl_fixtures['event']==gameweek) & (fpl_fixtures['home team']==team)]
        for fixture in home_fixtures.index:
            predictions.loc[player_ix, 'home'] = 1
            opponent = fpl_fixtures.loc[fixture, 'away team']
            # opponent xG per 90
            opponent_xG90 = team_stats.loc[opponent, 'xG'] / team_stats.loc[opponent,'MP']
            predictions.loc[player_ix, 'opponent xG per 90'] = opponent_xG90
            # opponent xGA per 90
            opponent_xGA90 = team_stats.loc[opponent, 'xGA'] / team_stats.loc[opponent,'MP']
            predictions.loc[player_ix, 'opponent xGA per 90'] = opponent_xGA90
            
            # predict player's points
            input_data = predictions.loc[player_ix:player_ix, input_columns]
            input_data = xgboost.DMatrix(input_data)
            prediction = model.predict(input_data)
            players.loc[player_ix, f'gameweek {gameweek} prediction'] += prediction
            
        # away game
        away_fixtures = fpl_fixtures[(fpl_fixtures['event']==gameweek) & (fpl_fixtures['away team']==team)]
        for fixture in away_fixtures.index:
            predictions.loc[player_ix, 'home'] = 0
            opponent = fpl_fixtures.loc[fixture, 'home team']
            # opponent xG per 90
            opponent_xG90 = team_stats.loc[opponent, 'xG'] / team_stats.loc[opponent,'MP']
            predictions.loc[player_ix, 'opponent xG per 90'] = opponent_xG90
            # opponent xGA per 90
            opponent_xGA90 = team_stats.loc[opponent, 'xGA'] / team_stats.loc[opponent,'MP']
            predictions.loc[player_ix, 'opponent xGA per 90'] = opponent_xGA90
            
            # predict player's points
            input_data = predictions.loc[player_ix:player_ix, input_columns]
            input_data = xgboost.DMatrix(input_data)
            prediction = model.predict(input_data)
            players.loc[player_ix, f'gameweek {gameweek} prediction'] += prediction

In [10]:
# all new predictions
players[output_columns]

,gameweek 39 prediction,gameweek 40 prediction,gameweek 41 prediction,gameweek 42 prediction,gameweek 43 prediction,gameweek 44 prediction,gameweek 45 prediction,gameweek 46 prediction,gameweek 47 prediction,gameweek 48 prediction
id,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
661,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
685,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
686,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
players[f'xPoints_next{horizon}'] = players[output_columns].sum(axis=1)

In [12]:
players.sort_values(f'gameweek {latest_gameweek+1} prediction', ascending=False)[['web_name','now_cost','adjusted points per game', f'gameweek {latest_gameweek+1} prediction']].head(20)

,web_name,now_cost,adjusted points per game,gameweek 39 prediction
id,,,,
1,Leno,45,2.658146,0.0
308,Almirón,50,2.252726,0.0
299,Lascelles,44,2.249018,0.0
300,Gayle,52,1.212500,0.0
301,Dummett,45,1.930903,0.0
302,Manquillo,39,2.257903,0.0
303,Krafth,43,2.699453,0.0
304,Murphy,46,2.259514,0.0
305,Schär,41,3.216583,0.0


In [13]:
players.sort_values(f'xPoints_next{horizon}', ascending=False)[['web_name','now_cost','adjusted points per game', f'xPoints_next{horizon}']].head(20)

,web_name,now_cost,adjusted points per game,xPoints_next10
id,,,,
1,Leno,45,2.658146,0.0
308,Almirón,50,2.252726,0.0
299,Lascelles,44,2.249018,0.0
300,Gayle,52,1.212500,0.0
301,Dummett,45,1.930903,0.0
302,Manquillo,39,2.257903,0.0
303,Krafth,43,2.699453,0.0
304,Murphy,46,2.259514,0.0
305,Schär,41,3.216583,0.0


In [14]:
# save data
filepath = Path('../data/fpl/data_week' + str(latest_gameweek) + str('.csv'))
players.to_csv(filepath)

# check shap-values for some particular predictions

In [ ]:
explainer = shap.TreeExplainer(model)
shap.initjs()

In [ ]:
# find player index
name_string = 'Salah'
playersFound = players[players['web_name'].str.contains(name_string)]
display(playersFound)

In [ ]:
player_ix = 233
print(players.loc[player_ix, 'web_name'])
print()
print(players.loc[player_ix, output_columns])

In [ ]:
#player_ix = 237
gameweek = latest_gameweek+1

team = players.loc[player_ix, 'team_name']

# home games
home_fixtures = fpl_fixtures[(fpl_fixtures['event']==gameweek) & (fpl_fixtures['home team']==team)]
for fixture in home_fixtures.index:
    predictions.loc[player_ix, 'home'] = 1
    opponent = fpl_fixtures.loc[fixture, 'away team']
    # opponent xG per 90
    opponent_xG90 = team_stats.loc[opponent, 'xG'] / team_stats.loc[opponent,'MP']
    predictions.loc[player_ix, 'opponent xG per 90'] = opponent_xG90
    # opponent xGA per 90
    opponent_xGA90 = team_stats.loc[opponent, 'xGA'] / team_stats.loc[opponent,'MP']
    predictions.loc[player_ix, 'opponent xGA per 90'] = opponent_xGA90

    # draw force plot
    input_data = predictions.loc[player_ix:player_ix, input_columns]
    shap_values = explainer.shap_values(input_data)

# away game
away_fixtures = fpl_fixtures[(fpl_fixtures['event']==gameweek) & (fpl_fixtures['away team']==team)]
for fixture in away_fixtures.index:
    predictions.loc[player_ix, 'home'] = 0
    opponent = fpl_fixtures.loc[fixture, 'home team']
    # opponent xG per 90
    opponent_xG90 = team_stats.loc[opponent, 'xG'] / team_stats.loc[opponent,'MP']
    predictions.loc[player_ix, 'opponent xG per 90'] = opponent_xG90
    # opponent xGA per 90
    opponent_xGA90 = team_stats.loc[opponent, 'xGA'] / team_stats.loc[opponent,'MP']
    predictions.loc[player_ix, 'opponent xGA per 90'] = opponent_xGA90

    # draw force plot
    input_data = predictions.loc[player_ix:player_ix, input_columns]
    shap_values = explainer.shap_values(input_data)

In [ ]:
print(players.loc[player_ix, 'web_name'])
print()
print(predictions.loc[player_ix])

In [ ]:
shap.force_plot(explainer.expected_value, shap_values, input_data)

In [ ]:
# salah
#shap.force_plot(explainer.expected_value, shap_values, input_data)

In [ ]:
# antonio
#shap.force_plot(explainer.expected_value, shap_values, input_data)